In [1]:
import collections
import pandas as pd
from tf_tools.load import load_tf
TF, API, A = load_tf(hoist=globals())

This is Text-Fabric 8.3.3
Api reference : https://annotation.github.io/text-fabric/cheatsheet.html

123 features found and 4 ignored
  0.00s loading features ...
   |     0.00s Dataset without structure sections in otext:no structure functions in the T-API
  6.21s All features loaded/computed - for details use loadLog()


In [30]:
genre_ranges = pd.read_csv('genre_ranges.csv')

In [31]:
genre_ranges

,book,ch start,vs start,ch end,vs end,genre
0,Genesis,1,1,4,16,prose
1,Genesis,4,17,4,23,list
2,Genesis,4,24,4,26,prose
3,Genesis,5,1,5,32,list
4,Genesis,6,1,9,29,prose
...,...,...,...,...,...,...
216,1_Chronicles,16,8,16,36,poetry
217,1_Chronicles,16,37,23,1,prose
218,1_Chronicles,23,2,27,34,list
219,1_Chronicles,28,1,29,30,prose


In [32]:
# check book values
genre_ranges.book.unique()

array(['Genesis', 'Exodus', 'Leviticus', 'Numbers', 'Deuteronomy',
       'Joshua', 'Judges', '1_Samuel', '2_Samuel', '1_Kings', '2_Kings',
       'Isaiah', 'Jeremiah', 'Ezekiel', 'Hosea', 'Joel', 'Amos',
       'Obadiah', 'Jonah', 'Micah', 'Nahum', 'Habakkuk', 'Zephaniah',
       'Haggai', 'Zechariah', 'Malachi', 'Psalms', 'Job', 'Proverbs',
       'Ruth', 'Song_of_songs', 'Ecclesiastes', 'Lamentations', 'Esther',
       'Daniel', 'Ezra', 'Nehemiah', '1_Chronicles', '2_Chronicles'],
      dtype=object)

In [33]:
# check genre values
genre_ranges.genre.unique()

array(['prose', 'list', 'poetry', 'instruction', 'prophetic'],
      dtype=object)

In [34]:
# check book name alignment with BHSA english names
for book in genre_ranges.book.unique():
    bhsa_node = T.nodeFromSection((book,))
    if not bhsa_node:
        raise Exception(book)

In [35]:
def verse_node_range(start, end, tf_api):
    """Generate a list of verse nodes for a given range of reference tuples.
    
    Note that start and end are both inclusive bounds.
    
    Args:
        start: 3-tuple of (book, n_ch, n_vs)
        end: 3-tuple of (book, n_ch, n_vs)
    Returns:
        list of nodes
    """
    start_node = tf_api.T.nodeFromSection(start)
    end_node = tf_api.T.nodeFromSection(end)
    nodes = [start_node]
    while nodes[-1] < end_node:
        nodes.append(tf_api.L.n(nodes[-1],'verse')[0])
    return nodes

In [36]:
# check for missing verses

verse2genre = {}

for book, startch, startvs, endch, endvs, genre in genre_ranges.values:
    start = (book, startch, startvs)
    end = (book, endch, endvs)
    for verse in verse_node_range(start, end, A.api):
        verse2genre[verse] = genre
        
len(verse2genre)

23213

In [37]:
all_verses = set(F.otype.s('verse'))

In [38]:
for missing_verse in (all_verses - set(verse2genre.keys())):
    print(missing_verse, T.sectionFromNode(missing_verse))

All verses are now accounted for!